In [1]:
from modeltraining import main
from peft import LoraConfig

rank = 64
peft_config = LoraConfig(r=rank, lora_alpha=rank*2, lora_dropout=0.05, target_modules=['value','query','key', 'dense'])
args_dict = {
    'model_path': 'sentence-transformers/all-MiniLM-L12-v1',
    'data_path': './dataset/data.csv',
    'device': 'cuda',
    'peft_config': peft_config,
    'batch_size': 4,
    'lr': 1e-5,
    'num_epochs': 20,
    'max_len': 200,
    'eval_every': 1000,
    'save_model_every': 1000,
    'save_model_path': './models/LoRa',
}

In [2]:
main(args_dict, use_argparse=False)

Arguments:
model_path: sentence-transformers/all-MiniLM-L12-v1
data_path: ./dataset/data.csv
device: cuda
peft_config: LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, base_model_name_or_path=None, revision=None, task_type=None, inference_mode=False, r=64, target_modules=['value', 'query', 'key', 'dense'], lora_alpha=128, lora_dropout=0.05, fan_in_fan_out=False, bias='none', modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None)
batch_size: 4
lr: 1e-05
num_epochs: 20
max_len: 200
eval_every: 1000
save_model_every: 1000
save_model_path: ./models/LoRa
triplet_loss: None
shuffle: True
eval_data_path: None
model_save_name: None
wandb_project_name: None
Loading model...
Loading Peft model From HuggingFace Hub...:  sentence-transformers/all-MiniLM-L12-v1
trainable params: 5357568 || all params: 38717568 || trainable%: 13.837563351086514


Creating triplets:   0%|          | 0/14 [00:00<?, ?group/s]

Initializing wandb...


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: ammarnasr. Use `wandb login --relogin` to force relogin


Training model...


...

Evaluating model


Generating embeddings:   0%|          | 0/257 [00:00<?, ?sentence/s]

Average inner_distance  within groups(The lower  the better): 0.24737105518579483
Average across_distance across groups(The higher the better): 0.8709536237376077
Accuracy: 94.55%
Saving model
Pushing model to hub


Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.bin:   0%|          | 0.00/21.5M [00:00<?, ?B/s]